In [6]:
#!pip install crewai pandas matplotlib seaborn openai ipywidgets

   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
    --------------------------------------- 0.3/11.5 MB ? eta -:--:--
    --------------------------------------- 0.3/11.5 MB ? eta -:--:--
    --------------------------------------- 0.3/11.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.5 MB 381.6 kB/s eta 0:00:29
   --- ------------------------------------ 1.0/11.5 MB 988.0 kB/s eta 0:00:11
   --- ------------------------------------ 1.0/11.5 MB 988.0 kB/s eta 0:00:11
   ------ --------------------------------- 1.8/11.5 MB 1.2 MB/s eta 0:00:09
   ---------- ----------------------------- 2.9/11.5 MB 1.8 MB/s eta 0:00:05
   -------------- ------------------------- 4.2/11.5 MB 2.3 MB/s eta 0:00:04
   ---------------- ----------------------- 4.7/11.5 MB 2.4 MB/s eta 0:00:03
   -------------------- ------------------- 6.0/11.5 MB 2.7 MB/s eta 0:00:03
   -------------------------

In [25]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output
from crewai import Agent, Task, Crew, LLM
from textwrap import dedent
import re

In [17]:
# List all CSVs in current directory
csv_files = [f for f in os.listdir() if f.endswith('.csv')]
file_picker = widgets.Dropdown(options=csv_files, description='📄 File:')

graph_options = widgets.SelectMultiple(
    options=['histogram', 'heatmap', 'boxplot'],
    value=['histogram', 'heatmap'],
    description='📊 Graphs:',
)

run_button = widgets.Button(description="▶️ Run CrewAI")

display(file_picker, graph_options, run_button)


Dropdown(description='📄 File:', options=('data.csv',), value='data.csv')

SelectMultiple(description='📊 Graphs:', index=(0, 1), options=('histogram', 'heatmap', 'boxplot'), value=('his…

Button(description='▶️ Run CrewAI', style=ButtonStyle())

In [28]:
def build_agents():
    reader = Agent(
        role="CSV Reader",
        goal="Read and summarize the CSV structure",
        backstory="You understand file formats and metadata.",
        verbose=True,
        llm=LLM(model="ollama/llama3.2", base_url="http://localhost:11434")
    )
    analyst = Agent(
        role="Data Analyst",
        goal="Extract meaningful insights from the dataset",
        backstory="You're a Python data analyst.",
        verbose=True,
        llm=LLM(model="ollama/llama3.2", base_url="http://localhost:11434")
    )
    viz = Agent(
        role="Visualization Creator",
        goal="Generate beautiful graphs from the dataset",
        backstory="You use matplotlib/seaborn for visualizations.",
        verbose=True,
        llm=LLM(model="ollama/llama3.2", base_url="http://localhost:11434")
    )
    return reader, analyst, viz

def build_tasks(file_name, graph_types, reader, analyst, viz):
    task1 = Task(
        description=dedent(f"""
            Load the CSV file '{file_name}'.
            Summarize it: number of rows, columns, column names, and types.
        """),
        expected_output="A short summary of the dataset's structure, including column names and data types.",
        agent=reader,
    )

    task2 = Task(
        description=dedent("""
            Analyze the dataset:
            - Show descriptive statistics
            - Discuss any notable patterns or correlations
        """),
        expected_output="Text summary of statistics, correlations, and possible trends.",
        agent=analyst,
        depends_on=[task1],
    )

    plots = ', '.join(graph_types)
    task3 = Task(
        description=dedent(f"""
            Based on the analysis, write Python code using matplotlib/seaborn to create these plots:
            {plots}.
            The output must be inside a ```python ... ``` block.
            Include `plt.show()` to render the plots.
        """),
        expected_output="Python code wrapped in triple backticks for generating the plots.",
        agent=viz,
        depends_on=[task2],
    )

    return [task1, task2, task3]


In [29]:
def extract_python_code(text):
    match = re.search(r"```python(.*?)```", text, re.DOTALL)
    return match.group(1).strip() if match else None

def run_generated_code(code):
    if code:
        try:
            print("🔧 Executing code...\n")
            exec_globals = {}
            exec(code, exec_globals)
        except Exception as e:
            print(f"❌ Error during execution: {e}")
    else:
        print("⚠️ No code found.")


In [30]:
# def on_run_button_click(b):
#     clear_output(wait=True)
#     display(file_picker, graph_options, run_button)

#     file_name = file_picker.value
#     graph_types = list(graph_options.value)

#     if not file_name:
#         print("❗ Please select a CSV file.")
#         return

#     print(f"📂 Selected file: {file_name}")
#     print(f"📊 Selected graphs: {', '.join(graph_types)}\n")

#     # Setup Crew
#     reader, analyst, viz = build_agents()
#     tasks = build_tasks(file_name, graph_types, reader, analyst, viz)
#     crew = Crew(agents=[reader, analyst, viz], tasks=tasks, verbose=True)

#     # Run
#     results = crew.kickoff()

#     # Extract + Execute Code
#     code_block = extract_python_code(results)
#     print("\n📄 Generated Code:\n")
#     print(code_block)
#     run_generated_code(code_block)

# run_button.on_click(on_run_button_click)


In [ ]:
csv_file = "data.csv"
while True:
    print("\n📊 Choose graphs to generate (comma-separated):")
    print("Options: histogram, heatmap, boxplot")
    selected = input("Your choice (or 'exit'): ").strip().lower()

    if selected in ['exit', 'quit']:
        print("👋 Exiting...")
        break

    # Parse and validate selection
    valid_graphs = ['histogram', 'heatmap', 'boxplot']
    chosen = [g for g in selected.replace(' ', '').split(',') if g in valid_graphs]

    if not chosen:
        print("⚠️ Invalid input. Try again.")
        continue

    print(f"\n✅ Running CrewAI for: {', '.join(chosen)}")

    # Agents + Tasks + Crew
    reader, analyst, viz = build_agents()
    tasks = build_tasks(csv_file, chosen, reader, analyst, viz)
    crew = Crew(agents=[reader, analyst, viz], tasks=tasks, verbose=True)

    # Run the agents
    results = crew.kickoff()
    
    # Extract code from final output
    final_output = results.final_output if hasattr(results, "final_output") else str(results)
    code = extract_python_code(final_output)
    
    print("\n📄 Generated Python Code:\n", code)
    run_generated_code(code)

    print("\n✅ Done. You can choose another graph or type 'exit'.")



📊 Choose graphs to generate (comma-separated):
Options: histogram, heatmap, boxplot


Your choice (or 'exit'):  heatmap



✅ Running CrewAI for: heatmap


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 5a323ef4-1f98-435c-be49-c3810e0a0713                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 6b88a956-ab24-4045-9fa6-cda6dd235238
       Status: Executing Task...

...

🚀 Crew: crew
├── 📋 Task: 7d05bc03-bdbc-4844-839e-1d2b088a1928
│      Assigned to: CSV Reader
│      Status: ✅ Completed
│   └── 🤖 Agent: CSV Reader
│           Status: ✅ Completed
├── 📋 Task: 2ae1418b-e121-4229-8c0f-9d4086083dd3
│      Assigned to: Data Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Data Analyst
│           Status: ✅ Completed
└── 📋 Task: 7ddc407c-fbdb-4d60-9dc6-354e4529806b
       Status: Executing Task...
    └── 🤖 Agent: Visualization Creator
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 7d05bc03-bdbc-4844-839e-1d2b088a1928
│      Assigned to: CSV Reader
│      Status: ✅ Completed
│   └── 🤖 Agent: CSV Reader
│           Status: ✅ Completed
├── 📋 Task: 2ae1418b-e121-4229-8c0f-9d4086083dd3
│      Assigned to: Data Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Data Analyst
│           Status: ✅ Completed
└── 📋 Task: 7ddc407c-fbdb-4d60-9dc6-354e4529806b
       Assigned to: Visualization Creator
       Status: ✅ Completed
    └── 🤖 Agent: Visualization Creator
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7ddc407c-fbdb-4d60-9dc6-354e4529806b                                                                     │
│  Agent: Visualization Creator                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 55f0cd05-8535-447e-9e3d-46a4f23ea77e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


📄 Generated Python Code:
 import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset into a DataFrame
data = {
    "age": [25, 30, 22, 35, 28, 40, 32, 45, 38, 50],
    "name": ["John", "Jane", "Jim", "Emily", "Michael", "Sarah", "William", "Olivia", "Alexander", "David"],
    "height": [1.75, 1.68, 1.82, 1.55, 1.80, 1.73, 1.85, 1.60, 1.83, 1.78],
    "weight": [70.5, 65.2, 75.0, 58.0, 72.8, 68.5, 77.3, 61.2, 76.9, 73.4]
}
df = pd.DataFrame(data)

# Calculate descriptive statistics
print("Descriptive Statistics:")
print(df.describe())

# Find correlations between variables
corr_matrix = df.corr()
print("\nCorrelation Matrix:")
print(corr_matrix)

# Plot a boxplot to visualize the distribution of each variable
plt.figure(figsize=(10,6))
sns.boxplot(data=df)
plt.title('Boxplot of Age, Height and Weight')
plt.show()

# Plot a 3D scatter plot to visualize the relationship between three variables
import matplotlib.pyplot as plt
from mpl_